可扩展性测试

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
from math import pow
import random

def fractal(branch, layer, prefix='0', reverse=False):
    nodes=[prefix]
    edges=[]
    if reverse:
        for i in range(layer-1):
            new_nodes=[]
            for j in nodes[i]:#为该层的每一个点添加下一层点
                for k in range(1,branch+1):
                    edges.append((j+str(k),j,1.0/pow(branch,i)*(1+(random.random()-0.5)*0.2)))
                    new_nodes.append(j+str(k))
            nodes.append(new_nodes)
    else:
        for i in range(layer-1):
            new_nodes=[]
            for j in nodes[i]:#为该层的每一个点添加下一层点
                for k in range(1,branch+1):
                    edges.append((j,j+str(k),1.0/pow(branch,i)*(1+(random.random()-0.5)*0.2)))
                    new_nodes.append(j+str(k))
            nodes.append(new_nodes)

    color=['tomato','orange','limegreen','c','dodgerblue','blueviolet','lightgray','peru']
    if reverse:
        color=color[::-1]
    c=[]
    for k,v in enumerate(nodes):
        for i in range(len(v)):
            c.append(color[k])
    return nodes,edges,c

## Creating a graph
做两个形状相同方向相反的分形网络，然后合并中心点。网络带20%的边权重噪声和节点权重噪声，因为SOM训练次数取决于数据分布，如果用节点嵌入完全重叠的理想网络测，聚类时间不真实

修改`fractal()`的参数，可以改变图的规模

In [3]:
G = nx.DiGraph()
nodes,edges,color1=fractal(3,3)
G.add_weighted_edges_from(edges)   
nodes,edges,color2=fractal(3,3,prefix='1',reverse=True)
G.add_node('1')
G.add_weighted_edges_from(edges)  
for k,v in G['0'].items():
    G.add_weighted_edges_from([('1',k,v['weight'])])
G.remove_node('0')

true_label=[len(i) if i[0]=='1' else len(i)*2 for i in G.nodes]

balance=[(random.random()-0.5)*0.2 for i in range(len(G.nodes))]
for i,node in enumerate(G.nodes):
    for k,v in G[node].items():
        balance[i]-=v['weight']
rev_G=G.reverse()
for i,node in enumerate(rev_G.nodes):
    for k,v in rev_G[node].items():
        balance[i]+=v['weight']

## RoleMagnet
这里给出测总时间的例子。如果要单独测表示学习或聚类的时间，需要修改rolemagnet.py，解除`mid_time=time.perf_counter()`的注释，把`mid_time`加到返回值

In [4]:
import rolemagnet as rm
import time

start=time.perf_counter()
vec,role,label=rm.role_magnet(G, balance=balance)
end=time.perf_counter()
print(len(G.nodes)*len(G.edges),end-start)

Embedding: 100.00%  -
SOM shape: [6, 8]
Training SOM: 125
600 2.5244376389309764


## Evaluation
用 Adjusted Rand Index 和 V-Measure 两种指标评价聚类结果

In [5]:
from sklearn.metrics.cluster import adjusted_rand_score, homogeneity_completeness_v_measure

print('Adjusted Rand Index:',adjusted_rand_score(true_label,label))
print('V-Measure:',homogeneity_completeness_v_measure(true_label,label))

Adjusted Rand Index: 1.0
V-Measure: (1.0, 1.0, 1.0)
